### Import modules

In [91]:
import pandas as pd

### Load data

In [92]:
df_genre = pd.read_csv('../data/raw/autotagging_genre.tsv', sep='\t')
df_instrument = pd.read_csv('../data/raw/autotagging_instrument.tsv', sep='\t')
df_moodtheme = pd.read_csv('../data/raw/autotagging_moodtheme.tsv', sep='\t')
df_meta = pd.read_csv('../data/raw/raw.meta.tsv', sep='\t')

In [93]:
df_genre.head()

,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS
0,track_0000214,artist_000014,album_000031,14/214.low.mp3,124.6,genre---punkrock
1,track_0000215,artist_000014,album_000031,15/215.low.mp3,151.4,genre---metal
2,track_0000216,artist_000014,album_000031,16/216.low.mp3,234.9,genre---metal
3,track_0000217,artist_000014,album_000031,17/217.low.mp3,127.9,genre---punkrock
4,track_0000218,artist_000014,album_000031,18/218.low.mp3,180.7,genre---punkrock


In [94]:
df_instrument.head()

,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS
0,track_0000382,artist_000020,album_000046,82/382.low.mp3,211.1,instrument---voice
1,track_0000383,artist_000020,album_000046,83/383.low.mp3,113.1,instrument---voice
2,track_0000384,artist_000020,album_000046,84/384.low.mp3,115.7,instrument---voice
3,track_0000386,artist_000020,album_000046,86/386.low.mp3,103.4,instrument---voice
4,track_0000387,artist_000020,album_000046,87/387.low.mp3,257.1,instrument---voice


In [95]:
df_moodtheme.head()

,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS
0,track_0000948,artist_000087,album_000149,48/948.low.mp3,212.7,mood/theme---background
1,track_0000950,artist_000087,album_000149,50/950.low.mp3,248.0,mood/theme---background
2,track_0000951,artist_000087,album_000149,51/951.low.mp3,199.7,mood/theme---background
3,track_0002165,artist_000326,album_000347,65/2165.low.mp3,229.0,mood/theme---film
4,track_0002263,artist_000320,album_000366,63/2263.low.mp3,494.7,mood/theme---melancholic


In [96]:
df_meta.head()

,TRACK_ID,ARTIST_ID,ALBUM_ID,TRACK_NAME,ARTIST_NAME,ALBUM_NAME,RELEASEDATE,URL
0,track_0000214,artist_000014,album_000031,Intro chiante,David TMX,Pouce !,2004-12-28,http://www.jamendo.com/track/214
1,track_0000215,artist_000014,album_000031,Aide toi...,David TMX,Pouce !,2004-12-28,http://www.jamendo.com/track/215
2,track_0000216,artist_000014,album_000031,Une charogne,David TMX,Pouce !,2004-12-28,http://www.jamendo.com/track/216
3,track_0000217,artist_000014,album_000031,Aimons nous...,David TMX,Pouce !,2004-12-28,http://www.jamendo.com/track/217
4,track_0000218,artist_000014,album_000031,Jouvencelle,David TMX,Pouce !,2004-12-28,http://www.jamendo.com/track/218


### Cleaning data

In [97]:
print(df_genre.isnull().sum())
print(df_instrument.isnull().sum())
print(df_moodtheme.isnull().sum())
print(df_meta.isnull().sum())

TRACK_ID     0
ARTIST_ID    0
ALBUM_ID     0
PATH         0
DURATION     0
TAGS         0
dtype: int64
TRACK_ID     0
ARTIST_ID    0
ALBUM_ID     0
PATH         0
DURATION     0
TAGS         0
dtype: int64
TRACK_ID     0
ARTIST_ID    0
ALBUM_ID     0
PATH         0
DURATION     0
TAGS         0
dtype: int64
TRACK_ID       0
ARTIST_ID      0
ALBUM_ID       0
TRACK_NAME     0
ARTIST_NAME    0
ALBUM_NAME     0
RELEASEDATE    0
URL            0
dtype: int64


In [98]:
df_genre = df_genre.drop_duplicates(subset=['TRACK_ID'])
df_instrument = df_instrument.drop_duplicates(subset=['TRACK_ID'])
df_moodtheme = df_moodtheme.drop_duplicates(subset=['TRACK_ID'])
df_meta = df_meta.drop_duplicates(subset=['TRACK_ID'])

### Transform tags columns

In [99]:
def genre_tags_to_list(tags):
    if isinstance(tags, str):
        return [tag.strip() for tag in tags.split('genre---') if tag.strip()]
    return []

def instrument_tags_to_list(tags):
    if isinstance(tags, str):
        return [tag.strip() for tag in tags.split('instrument---') if tag.strip()]
    return []

def moodtheme_tags_to_list(tags):
    if isinstance(tags, str):
        return [tag.strip() for tag in tags.split('mood/theme---') if tag.strip()]
    return []

In [100]:
df_genre['GENRES'] = df_genre['TAGS'].apply(genre_tags_to_list)
df_instrument['INSTRUMENTS'] = df_instrument['TAGS'].apply(instrument_tags_to_list)
df_moodtheme['MOOD_THEME'] = df_moodtheme['TAGS'].apply(moodtheme_tags_to_list)

### Merge dataframes

In [141]:
df_meta_selected = df_meta.drop(columns=['URL'])
df_genre_selected = df_genre[['TRACK_ID', 'PATH', 'DURATION', 'GENRES']]
df_instrument_selected = df_instrument[['TRACK_ID', 'INSTRUMENTS']]
df_moodtheme_selected = df_moodtheme[['TRACK_ID', 'MOOD_THEME']]

df_merged = pd.merge(df_meta_selected, df_genre_selected, on='TRACK_ID', how='left')
df_merged = pd.merge(df_merged, df_instrument_selected, on='TRACK_ID', how='left')
df_merged = pd.merge(df_merged, df_moodtheme_selected, on='TRACK_ID', how='left')

In [142]:
print(df_merged.isnull().sum())

TRACK_ID           0
ARTIST_ID          0
ALBUM_ID           0
TRACK_NAME         0
ARTIST_NAME        0
ALBUM_NAME         0
RELEASEDATE        0
PATH            1424
DURATION        1424
GENRES          1424
INSTRUMENTS    31496
MOOD_THEME     38153
dtype: int64


In [143]:
df_merged = df_merged.dropna(subset=['PATH'])

In [144]:
print(df_merged.isnull().sum())

TRACK_ID           0
ARTIST_ID          0
ALBUM_ID           0
TRACK_NAME         0
ARTIST_NAME        0
ALBUM_NAME         0
RELEASEDATE        0
PATH               0
DURATION           0
GENRES             0
INSTRUMENTS    30313
MOOD_THEME     36960
dtype: int64


In [146]:
df_merged['INSTRUMENTS'] = df_merged['INSTRUMENTS'].apply(lambda x: x if isinstance(x, list) else ['unknown'])
df_merged['MOOD_THEME'] = df_merged['MOOD_THEME'].apply(lambda x: x if isinstance(x, list) else ['unknown'])

In [147]:
print(df_merged.isnull().sum())

TRACK_ID       0
ARTIST_ID      0
ALBUM_ID       0
TRACK_NAME     0
ARTIST_NAME    0
ALBUM_NAME     0
RELEASEDATE    0
PATH           0
DURATION       0
GENRES         0
INSTRUMENTS    0
MOOD_THEME     0
dtype: int64
